### GFMS:
http://flood.umd.edu  
http://eagle2.umd.edu/flood/download/  subfolder: [year][yearmm]  
Flood_byStor_yyyymmddhh.bin  Flood intensity (in depth) above threshold mm  

In [7]:
from datetime import date
import requests, wget
import os, sys

In [8]:
baseurl = "http://eagle2.umd.edu/flood/download/"
# find the latest data set
cur_year, cur_month = map(str,[date.today().year,date.today().month])
cur_month = cur_month.zfill(2)
dataurl = baseurl + cur_year + "/" + cur_year + cur_month 
print(dataurl)

http://eagle2.umd.edu/flood/download/2020/202001


In [9]:
response = requests.get(dataurl)
raw_text = response.text.split()
data_list = [x.split("'")[1] for x in raw_text if "href" in x]
latest_data = data_list[-2]
latest_data_url = dataurl + "/" + latest_data
print(latest_data_url)

http://eagle2.umd.edu/flood/download/2020/202001/Flood_byStor_2020011118.bin


In [10]:
if not os.path.exists(latest_data):
    wget.download(latest_data_url)

In [21]:
# generate header file
hdr_header = """NCOLS 2458
NROWS 800
XLLCORNER -127.25
YLLCORNER -50
CELLSIZE 0.125
PIXELTYPE FLOAT
BYTEORDER LSBFIRST
NODATA_VALUE -9999
"""
header_file = latest_data.replace(".bin",".hdr")
with open(header_file,"w") as f:
    f.write(hdr_header)

In [22]:
vrt_template="""<VRTDataset rasterXSize="2458" rasterYSize="800">
  <SRS>GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]]</SRS>
  <GeoTransform> -1.2725000000000000e+02,  1.2500000000000000e-01,  0.0000000000000000e+00,  5.0000000000000000e+01,  0.0000000000000000e+00, -1.2500000000000000e-01</GeoTransform>
  <VRTRasterBand dataType="Float32" band="1">
    <Metadata>
      <MDI key="STATISTICS_APPROXIMATE">YES</MDI>
      <MDI key="STATISTICS_MAXIMUM">1345.408203125</MDI>
      <MDI key="STATISTICS_MEAN">26.161176808621</MDI>
      <MDI key="STATISTICS_MINIMUM">1.1765951057896e-07</MDI>
      <MDI key="STATISTICS_STDDEV">120.73468295071</MDI>
      <MDI key="STATISTICS_VALID_PERCENT">1.117</MDI>
    </Metadata>
    <NoDataValue>-9999</NoDataValue>
    <ComplexSource>
      <SourceFilename relativeToVRT="1">{}</SourceFilename>
      <SourceBand>1</SourceBand>
      <SourceProperties RasterXSize="2458" RasterYSize="800" DataType="Float32" BlockXSize="2458" BlockYSize="1" />
      <SrcRect xOff="0" yOff="0" xSize="2458" ySize="800" />
      <DstRect xOff="0" yOff="0" xSize="2458" ySize="800" />
      <NODATA>-9999</NODATA>
    </ComplexSource>
  </VRTRasterBand>
</VRTDataset>"""

In [23]:
# generate VRT file
vrt_file = latest_data.replace(".bin",".vrt")
with open(vrt_file,"w") as f:
    f.write(vrt_template.format(latest_data))

In [24]:
!gdalinfo {vrt_file}

Driver: VRT/Virtual Raster
Files: Flood_byStor_2020011118.vrt
       Flood_byStor_2020011118.bin
Size is 2458, 800
Coordinate System is:
GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["WGS 84",6378137,298.257223563,
            AUTHORITY["EPSG","7030"]],
        AUTHORITY["EPSG","6326"]],
    PRIMEM["Greenwich",0,
        AUTHORITY["EPSG","8901"]],
    UNIT["degree",0.0174532925199433,
        AUTHORITY["EPSG","9122"]],
    AUTHORITY["EPSG","4326"]]
Origin = (-127.250000000000000,50.000000000000000)
Pixel Size = (0.125000000000000,-0.125000000000000)
Corner Coordinates:
Upper Left  (-127.2500000,  50.0000000) (127d15' 0.00"W, 50d 0' 0.00"N)
Lower Left  (-127.2500000, -50.0000000) (127d15' 0.00"W, 50d 0' 0.00"S)
Upper Right ( 180.0000000,  50.0000000) (180d 0' 0.00"E, 50d 0' 0.00"N)
Lower Right ( 180.0000000, -50.0000000) (180d 0' 0.00"E, 50d 0' 0.00"S)
Center      (  26.3750000,   0.0000000) ( 26d22'30.00"E,  0d 0' 0.01"N)
Band 1 Block=128x128 Type=Float32, ColorInterp=Undefin